In [1]:
import pandas as pd
import numpy as np

In [2]:
import random
random.seed(13)

### Import data

In [3]:
# maybe use np.where to reduce the data imported

Read

In [4]:
code = pd.read_csv('data/coding7128.tsv', sep='\t')

In [5]:
# need to import this file in slightly different way because of different delimiters
clinical = pd.read_csv('data/covid19_clinical_sample.txt',sep='\t|,', engine = 'python')

In [6]:
clinical_old = pd.read_csv('data/tpp_gp_clinical_subset.csv')

In [7]:
ic10 = pd.read_csv('data/ic10_lookup.csv')

In [8]:
GPic10 = pd.read_csv('data/GP_ic10_lookup.csv') 

In [9]:
# need to import this file in slightly different way because of different delimiters
scripts = pd.read_csv('data/covid19_prescriptions_sample.txt', sep='\t|,', engine = 'python')

In [10]:
scripts_old = pd.read_csv('data/sample_covid19_tpp_gp_scripts.txt')

have a look at the data

In [11]:
code.head()
# CTV3 codes (used by GPs, in dataset clinical) and the corresponding disease and incidences
# code can also be measurements, but will not be used by us

,coding,meaning
0,-1,redacted - potentially sensitive or identifying
1,-2,redacted - rare occupation
2,.....,Read thesaurus
3,.0...,See 0....
4,.01..,See 01...


In [12]:
clinical
# GP dataset from every patient
# each line corresponds to one incident with a date (event_dt)
# eid: patient number 
# code_type: ?
# value: measured value of a test or whatever

,Unnamed: 0,eid,event_dt,code_type,code,value
0,0,1000034,25/07/2017,0,XaIyt,0.000000
1,1,1000034,26/06/2018,0,XaF8d,0.000000
2,2,1000034,22/08/2017,0,XaF8d,0.000000
3,3,1000034,04/08/2015,0,XaF8d,0.000000
4,4,1000034,07/06/2018,0,XaIeG,0.000000
...,...,...,...,...,...,...
999995,999995,1033328,23/05/2012,0,42N..,0.330000
999996,999996,1033328,10/07/2013,0,246A.,80.000000
999997,999997,1033328,23/05/2012,0,XaK8y,83.300003
999998,999998,1033328,20/05/1997,0,246A.,80.000000


In [13]:
clinical_old

,Unnamed: 0,eid,event_dt,code_type,code,value
0,101084316,4392480,31/12/2020,0,XaERu,11.00
1,9359934,1311969,31/12/2020,0,XaN3z,0.80
2,9360007,1311969,31/12/2020,0,44P6.,0.58
3,111269383,4731119,31/12/2020,0,XaIm8,0.00
4,9360002,1311969,31/12/2020,0,44P5.,1.30
...,...,...,...,...,...,...
49995,46497795,2564395,31/12/2019,0,XaFqt,0.00
49996,46497794,2564395,31/12/2019,0,XaIVg,0.00
49997,41639202,2400982,31/12/2019,0,XaEJK,0.00
49998,116371391,4900832,31/12/2019,0,X1004,0.00


In [14]:
ic10
# description of ICD20 codes

,Unnamed: 0,ICD10_CODE,ALT_CODE,USAGE,USAGE_UK,DESCRIPTION,MODIFIER_4,MODIFIER_5,QUALIFIERS,GENDER_MASK,MIN_AGE,MAX_AGE,TREE_DESCRIPTION
0,0,A00,A00,DEFAULT,3.0,Cholera,,,,,,,
1,1,A00.0,A000,DEFAULT,3.0,"Cholera due to Vibrio cholerae 01, biovar chol...",,,,,,,
2,2,A00.1,A001,DEFAULT,3.0,"Cholera due to Vibrio cholerae 01, biovar eltor",,,,,,,
3,3,A00.9,A009,DEFAULT,3.0,"Cholera, unspecified",,,,,,,
4,4,A01,A01,DEFAULT,3.0,Typhoid and paratyphoid fevers,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17931,17931,U84.8,U848,DEFAULT,3.0,Resistance to other specified antimicrobial drug,,,,,,,
17932,17932,U84.9,U849,DEFAULT,3.0,Resistance to unspecified antimicrobial drugs,,,,,,,
17933,17933,U85,U85X,DEFAULT,3.0,Resistance to antineoplastic drugs,,,,,,,
17934,17934,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
GPic10
#translates CTV3 codes (read_code) to ICD10 (icd10_code)

,Unnamed: 0,read_code,icd10_code,mapping_status,refine_flag,add_code_flag,element_num,block_num
0,0,123..,Z831,D,C,M,0.0,0.0
1,1,123..,Z830,A,C,M,0.0,0.0
2,2,1231.,Z831,G,C,M,0.0,0.0
3,3,1232.,Z831,G,C,M,0.0,0.0
4,4,1233.,Z831,G,C,M,0.0,0.0
...,...,...,...,...,...,...,...,...
116372,116372,ZVy..,Z769,D,C,C,0.0,0.0
116373,116373,ZVz..,Z769,D,C,C,0.0,0.0
116374,116374,NaN,NaN,NaN,NaN,NaN,NaN,NaN
116375,116375,"Contains information from NHS Digital, licence...",NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
scripts
# all medication patients got from their GP, incl. the COVID-19 vaccine 
# needs to be complete if want to compare vaccinated vs. non-vaccinated, but seems like not all vaccinations registered here but just the ones done by the GP

,Unnamed: 0,eid,issue_date,dmd_code
0,0,1000034,05/08/2016,317310006
1,1,1000034,16/03/2015,317310006
2,2,1000034,22/05/2017,317310006
3,3,1000034,31/03/2016,317310006
4,4,1000034,23/01/2019,317310006
...,...,...,...,...
999995,999995,1069267,22/06/2015,347928005
999996,999996,1069267,26/07/1997,382311000001103
999997,999997,1069267,13/03/1997,374296008
999998,999998,1069267,15/07/1997,382311000001103


In [17]:
scripts_old

,Unnamed: 0,eid,issue_date,dmd_code
0,39278003,3743492,2021-03-22,320031002
1,35914327,3508920,2021-03-15,3591211000001105
2,49790799,4472075,2021-05-03,33405511000001102
3,31447155,3192040,2021-03-15,321745007
4,69964306,5879124,2021-01-19,320031002
...,...,...,...,...
99995,62996879,5393814,2021-01-27,320030001
99996,47834233,4333940,2021-01-03,38893411000001105
99997,33831040,3365447,2021-03-31,318052005
99998,12832421,1889162,2021-03-30,321152004


In [18]:
# get few individuals from large dataset -> may try to call only part of scripts dataset, where code == one of covid vaccine

# try only to get lines from scripts dataset where dmd_code== a vaccine 
# use function read_csv. only where irgendwas

print('lines in samplescripts: ', len(scripts))
print('number of subjects in samplescripts: ', len(scripts['eid'].value_counts()))

lines in samplescripts:  1000000
number of subjects in samplescripts:  2589


### Preparing the data

#### Adding column with IC10 categories to clinical dataset for simpler analysis later on

In [19]:
# create a reduced dataset for merging with the clinical dataset 
GPic10_red =  GPic10[['read_code', 'icd10_code']]
# rename to merge ont he ctv3 column with the clinical dataset
GPic10_red = GPic10_red.rename(columns = {'read_code':'code'})
GPic10_red.head()

,code,icd10_code
0,123..,Z831
1,123..,Z830
2,1231.,Z831
3,1232.,Z831
4,1233.,Z831


In [20]:
clinical_ic10 = pd.merge(clinical, GPic10_red, how='inner', on='code')
clinical_ic10

,Unnamed: 0,eid,event_dt,code_type,code,value,icd10_code
0,42,1000034,13/05/1998,0,XE1lT,0.0,S832
1,326324,1011402,15/08/2003,0,XE1lT,0.0,S832
2,326338,1011402,11/03/2004,0,XE1lT,0.0,S832
3,326339,1011402,16/07/2004,0,XE1lT,0.0,S832
4,371890,1012829,06/01/2015,0,XE1lT,0.0,S832
...,...,...,...,...,...,...,...
188488,996399,1033266,23/06/2015,0,XaDbE,0.0,R568
188489,999941,1033328,11/12/2018,0,X2060,0.0,I839
188490,999941,1033328,11/12/2018,0,X2060,0.0,I83
188491,999941,1033328,11/12/2018,0,X2060,0.0,O220


#### Data Cleaning

In [21]:
# drop unnecessary columns
clinical_ic10.drop(columns = ['code_type', 'value', 'Unnamed: 0'], inplace = True)
clinical_ic10

,eid,event_dt,code,icd10_code
0,1000034,13/05/1998,XE1lT,S832
1,1011402,15/08/2003,XE1lT,S832
2,1011402,11/03/2004,XE1lT,S832
3,1011402,16/07/2004,XE1lT,S832
4,1012829,06/01/2015,XE1lT,S832
...,...,...,...,...
188488,1033266,23/06/2015,XaDbE,R568
188489,1033328,11/12/2018,X2060,I839
188490,1033328,11/12/2018,X2060,I83
188491,1033328,11/12/2018,X2060,O220


In [22]:
pd.isnull(clinical_ic10).sum()
# no NaN

eid           0
event_dt      0
code          0
icd10_code    0
dtype: int64

In [23]:
pd.isnull(ic10).sum()

Unnamed: 0          0
ICD10_CODE          1
ALT_CODE            2
USAGE               2
USAGE_UK            2
DESCRIPTION         2
MODIFIER_4          2
MODIFIER_5          2
QUALIFIERS          2
GENDER_MASK         2
MIN_AGE             2
MAX_AGE             2
TREE_DESCRIPTION    2
dtype: int64

In [24]:
# remove duplicate entries in clinical dataset 
# (for other datasets, it should not be a problem)
clinical_ic10.drop_duplicates(inplace = True) 

# remove NaN in selected column
ic10.dropna(subset=['DESCRIPTION'], inplace = True)

In [25]:
pd.isnull(ic10).sum()

Unnamed: 0          0
ICD10_CODE          0
ALT_CODE            0
USAGE               0
USAGE_UK            0
DESCRIPTION         0
MODIFIER_4          0
MODIFIER_5          0
QUALIFIERS          0
GENDER_MASK         0
MIN_AGE             0
MAX_AGE             0
TREE_DESCRIPTION    0
dtype: int64

In [26]:
clinical_ic10

,eid,event_dt,code,icd10_code
0,1000034,13/05/1998,XE1lT,S832
1,1011402,15/08/2003,XE1lT,S832
2,1011402,11/03/2004,XE1lT,S832
3,1011402,16/07/2004,XE1lT,S832
4,1012829,06/01/2015,XE1lT,S832
...,...,...,...,...
188488,1033266,23/06/2015,XaDbE,R568
188489,1033328,11/12/2018,X2060,I839
188490,1033328,11/12/2018,X2060,I83
188491,1033328,11/12/2018,X2060,O220


In [27]:
type(('O','P','S','T','U','V','W','X','Y','Z'))

tuple

In [28]:
# remove not considered clinical events in the clinical dataset (e.g. external factors)

""""
Monica's suggestions: 
remove O-Z (pregnancy, congenital, others, COVID-19, external causes, factors)
maybe keep Q (congenital) as underlying disease, keep all others

Renée:
I would also keep R in there: "In general, categories in this chapter include the less well-defined conditions and symptoms that, without the necessary study of the case to establish a final diagnosis, point perhaps equally to two or more diseases or to two or more systems of the body.

Decision:
O-Z remove, but leave R as adverse event & Q as underlying disease.

"""

# tuple of categories we do not consider as adverse event or underlying disease:
# the ones starting with O-Z (except Q and R)
exclude = ('O','P','S','T','U','V','W','X','Y','Z')

# broadcasting the clinical dataset for all icd codes except those we exclude
clinical_ic10 = clinical_ic10[~clinical_ic10['icd10_code'].astype(str).str.startswith(exclude)]
clinical_ic10

,eid,event_dt,code,icd10_code
12,1000034,01/01/1978,XE0e7,N419
13,1000034,01/01/1978,XE0e7,N41
14,1013867,10/01/2003,XE0e7,N419
15,1013867,10/01/2003,XE0e7,N41
16,1000034,23/01/2019,XE1FC,M543
...,...,...,...,...
188460,1033266,11/02/2016,XaQv0,R452
188461,1033266,25/02/2015,X75Z0,R568
188488,1033266,23/06/2015,XaDbE,R568
188489,1033328,11/12/2018,X2060,I839


### Find who is vaccinated

The meaning of the SNOMED codes ('dmd_code' in the 'scripts' dataset, where medications that the patients got are listed) were found on: https://services.nhsbsa.nhs.uk/dmd-browser/amp/view/159421?ref=YW1wTmFtZT1jb3ZpZCZzZWFyY2hUeXBlPUFNUCZzaG93SW52YWxpZEl0ZW1zPWZhbHNl#ampCodes

Each vaccine has a specific code.

In [29]:
# broadcasting to find individuals with AstraZEneca vaccine
mask_AZ = scripts['dmd_code']== 39114911000001105
scripts[mask_AZ].head()

,Unnamed: 0,eid,issue_date,dmd_code
261135,261135,1018525,21/03/2021,39114911000001105
597692,597692,1040468,30/01/2021,39114911000001105
970763,970763,1066390,29/01/2021,39114911000001105
970765,970765,1066390,20/03/2021,39114911000001105


In [30]:
# getting IDs of individuals with any corona vaccine first (compare vaccinates vs. not)
# may look at effects of specific vaccines later
dict_vac = {39326911000001101: 'Moderna', 39230211000001104: 'Janssen', \
            39826711000001101:'Medicago', 39473011000001103: 'Baxter', \
            39114911000001105: 'AstraZeneca', 39115611000001103: 'Pfizer',\
            39373511000001104: 'Valneva'}

for dmd_code in dict_vac:
    mask_vac = scripts['dmd_code']== dmd_code
    print(dict_vac[dmd_code], ':',len(scripts[mask_vac]))

Moderna : 0
Janssen : 0
Medicago : 0
Baxter : 0
AstraZeneca : 4
Pfizer : 2
Valneva : 0


In [31]:
# mask for any vaccine
mask_vac = scripts['dmd_code'].isin(dict_vac)

# number of vaccinated people
print('Number of vaccinated people:', len(scripts[mask_vac]))

Number of vaccinated people: 6


In [32]:
# finding eids of vaccinated people in the dataset
id_vac = scripts[mask_vac]['eid'] 
id_vac

19471     1000899
261135    1018525
466220    1031590
597692    1040468
970763    1066390
970765    1066390
Name: eid, dtype: int64

In [33]:
# finding events of the vaccinated people, but only events before vaccination found 
# -> may need a more recent dataset
# need to separate clinical events before & after vaccine
mask_vac_clin = clinical_ic10['eid'].isin(id_vac)
clinical_ic10[mask_vac_clin] 

,eid,event_dt,code,icd10_code
2099,1000899,29/07/2013,X75rv,M255
2100,1000899,29/07/2013,X75rv,A180D
2101,1000899,29/07/2013,X75rv,M011A
2104,1000899,06/02/2020,X75rv,M255
2105,1000899,06/02/2020,X75rv,A180D
...,...,...,...,...
184032,1018525,30/03/2020,K24..,N459
184033,1018525,30/03/2020,K24..,N45
184034,1018525,22/03/2012,J323z,K429
184035,1018525,14/03/1995,H03z.,J039


# categorize event as before or after COVID-19 vaccination 

### make 'event_date' in 'clinical' be interpreted as a date

In [34]:
clinical_ic10.head(10)

,eid,event_dt,code,icd10_code
12,1000034,01/01/1978,XE0e7,N419
13,1000034,01/01/1978,XE0e7,N41
14,1013867,10/01/2003,XE0e7,N419
15,1013867,10/01/2003,XE0e7,N41
16,1000034,23/01/2019,XE1FC,M543
17,1000034,23/01/2019,XE1FC,M544
18,1000034,23/01/2019,XE1FC,G570
19,1000034,23/01/2019,XE1FC,M511
20,1000034,23/01/2019,XE1FC,M511D
21,1000034,23/01/2019,XE1FC,G551A


In [35]:
from datetime import datetime, timedelta

### only to understand the function, can be deleted later

In [36]:
clinical_ic10['event_dt'].unique()
# find out if the other dates are e.g. 06/01/2020 or 6/1/2020

array(['01/01/1978', '10/01/2003', '23/01/2019', ..., '18/11/1996',
       '20/02/1957', '04/12/1956'], dtype=object)

In [37]:
example_date = clinical_ic10['event_dt'].unique()[0]

In [38]:
example_date_new = datetime.strptime(example_date, '%d/%m/%Y')
example_date_new #2020

datetime.datetime(1978, 1, 1, 0, 0)

In [39]:
example_date_2 = clinical_ic10['event_dt'].unique()[1]
example_date_2 

'10/01/2003'

In [40]:
example_date_2_new = datetime.strptime(example_date_2, '%d/%m/%Y')
example_date_2_new #2019

datetime.datetime(2003, 1, 10, 0, 0)

In [41]:
# compare the 2 dates
example_date_new > example_date_2_new # 2020 > 2019 --> True
# recent > older --> True

False

### actually convert the column into the before/after

In [42]:
# convert entire event_dt in datetime format
clinical_ic10['event_dt_date_format'] = pd.to_datetime(clinical_ic10['event_dt'])
clinical_ic10

C:\Users\moniz\AppData\Local\Temp/ipykernel_7584/1462185986.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clinical_ic10['event_dt_date_format'] = pd.to_datetime(clinical_ic10['event_dt'])


,eid,event_dt,code,icd10_code,event_dt_date_format
12,1000034,01/01/1978,XE0e7,N419,1978-01-01
13,1000034,01/01/1978,XE0e7,N41,1978-01-01
14,1013867,10/01/2003,XE0e7,N419,2003-10-01
15,1013867,10/01/2003,XE0e7,N41,2003-10-01
16,1000034,23/01/2019,XE1FC,M543,2019-01-23
...,...,...,...,...,...
188460,1033266,11/02/2016,XaQv0,R452,2016-11-02
188461,1033266,25/02/2015,X75Z0,R568,2015-02-25
188488,1033266,23/06/2015,XaDbE,R568,2015-06-23
188489,1033328,11/12/2018,X2060,I839,2018-11-12


In [43]:
# checking the date range
clinical_ic10.sort_values(by = 'event_dt_date_format', ascending = False)

,eid,event_dt,code,icd10_code,event_dt_date_format
173433,1014125,07/07/2037,X785s,Q613,2037-07-07
183184,1016438,12/03/2021,X2033,I272,2021-12-03
183185,1016438,12/03/2021,X2033,I270,2021-12-03
160773,1028939,12/03/2021,XE0r3,R198,2021-12-03
121248,1028594,12/03/2021,XaE5D,M858,2021-12-03
...,...,...,...,...,...
81833,1000624,01/01/1901,M12z0,L309,1901-01-01
186024,1023271,01/01/1901,XaIRZ,J303,1901-01-01
82616,1009699,01/01/1901,G2...,I10X,1901-01-01
188015,1031708,01/01/1901,A741.,B330,1901-01-01


In [44]:
# only to understand the function, can be deleted later!!
# compare 2 dates (first row and last row) from the new column 

print(clinical_ic10['event_dt_date_format'].iloc[-1]) # 2019
print(clinical_ic10['event_dt_date_format'].iloc[0]) # 2020
clinical_ic10['event_dt_date_format'].iloc[0] > clinical_ic10['event_dt_date_format'].iloc[-1]

2018-11-12 00:00:00
1978-01-01 00:00:00


False

### subset 'scripts' to only those patients (eid) that got the COVID-19 vaccine and drop 'Unnamed: 0' column

In [45]:
scripts.drop(columns = ['Unnamed: 0'], inplace = True)
mask_scripts_vac = scripts['dmd_code'].isin(dict_vac)
scripts_vaccines_only = scripts[mask_scripts_vac] 

len(scripts[mask_scripts_vac]) # corresponds to finding above

6

In [46]:
scripts_vaccines_only.head() # contains eid and date of vaccination
# length: 30

,eid,issue_date,dmd_code
19471,1000899,09/01/2021,39115611000001103
261135,1018525,21/03/2021,39114911000001105
466220,1031590,23/02/2021,39115611000001103
597692,1040468,30/01/2021,39114911000001105
970763,1066390,29/01/2021,39114911000001105


In [47]:
clinical_ic10.head()
# length: 4156
len(clinical_ic10['eid'].unique()) # some eid are in there multiple times (of course)

1238

### add a column with the date of vaccination to clinical_ic10

In [48]:
# merge clinical_ic10 and scripts_vaccines_only to add the issue_date (date of vaccine) to the
# Monica's suggestion: merge 'outer' to also include people that got vaccinated but have no medical events noted
clinical_vac = pd.merge(clinical_ic10, scripts_vaccines_only, how ='outer', on ='eid')

In [49]:
clinical_vac

,eid,event_dt,code,icd10_code,event_dt_date_format,issue_date,dmd_code
0,1000034,01/01/1978,XE0e7,N419,1978-01-01,NaN,NaN
1,1000034,01/01/1978,XE0e7,N41,1978-01-01,NaN,NaN
2,1000034,23/01/2019,XE1FC,M543,2019-01-23,NaN,NaN
3,1000034,23/01/2019,XE1FC,M544,2019-01-23,NaN,NaN
4,1000034,23/01/2019,XE1FC,G570,2019-01-23,NaN,NaN
...,...,...,...,...,...,...,...
133024,1027425,12/07/2012,J090.,E52XD,2012-12-07,NaN,NaN
133025,1027425,12/07/2012,J090.,K938A,2012-12-07,NaN,NaN
133026,1040468,NaN,NaN,NaN,NaT,30/01/2021,3.911491e+16
133027,1066390,NaN,NaN,NaN,NaT,29/01/2021,3.911491e+16


In [50]:
len(clinical_vac) 
# total amounts of events: vaccinations and medical events from clinical_ic10

133029

In [51]:
clinical_vac['issue_date'].isna().sum()
# 4150 events of unvaccinated, 34 of vaccinated patients

132761

### add column in clinical_vac that says whether a patient is vaccinated or not

In [52]:
mask_vaccinated = clinical_vac['eid'].isin(id_vac)
clinical_vac['vaccinated'] = mask_vaccinated

# check how many vaccinated people there are
clinical_vac['vaccinated'].value_counts()
# 34 -> correct

# clinical_ic10['eid'].loc[clinical_ic10['vaccinated']==True]

False    132761
True        268
Name: vaccinated, dtype: int64

In [53]:
clinical_vac
# rows with event_dt=NaN: vaccinated eids with medical events
# maybe rename it

,eid,event_dt,code,icd10_code,event_dt_date_format,issue_date,dmd_code,vaccinated
0,1000034,01/01/1978,XE0e7,N419,1978-01-01,NaN,NaN,False
1,1000034,01/01/1978,XE0e7,N41,1978-01-01,NaN,NaN,False
2,1000034,23/01/2019,XE1FC,M543,2019-01-23,NaN,NaN,False
3,1000034,23/01/2019,XE1FC,M544,2019-01-23,NaN,NaN,False
4,1000034,23/01/2019,XE1FC,G570,2019-01-23,NaN,NaN,False
...,...,...,...,...,...,...,...,...
133024,1027425,12/07/2012,J090.,E52XD,2012-12-07,NaN,NaN,False
133025,1027425,12/07/2012,J090.,K938A,2012-12-07,NaN,NaN,False
133026,1040468,NaN,NaN,NaN,NaT,30/01/2021,3.911491e+16,True
133027,1066390,NaN,NaN,NaN,NaT,29/01/2021,3.911491e+16,True


### add random issue date to events of unvaccinated patients to 'issue_date'  in clinical_vac 

In [54]:
# make list from 'issue_date' in scripts_vaccines_only
issue_dates = scripts_vaccines_only['issue_date'].tolist()
issue_dates

['09/01/2021',
 '21/03/2021',
 '23/02/2021',
 '30/01/2021',
 '29/01/2021',
 '20/03/2021']

In [55]:
random.choice(issue_dates)

'23/02/2021'

In [56]:
# potential alternative: use np.select()

# randomly sample a date from issue_dates for each unvaccinated patient

for i in range (len(clinical_vac['eid'])):
    if clinical_vac['vaccinated'].loc[i] == False:
        clinical_vac['issue_date'].loc[i] = random.choice(issue_dates) # we just want the values attribute, not in an array
        
clinical_vac

C:\Users\moniz\AppData\Roaming\Python\Python38\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,eid,event_dt,code,icd10_code,event_dt_date_format,issue_date,dmd_code,vaccinated
0,1000034,01/01/1978,XE0e7,N419,1978-01-01,23/02/2021,NaN,False
1,1000034,01/01/1978,XE0e7,N41,1978-01-01,20/03/2021,NaN,False
2,1000034,23/01/2019,XE1FC,M543,2019-01-23,20/03/2021,NaN,False
3,1000034,23/01/2019,XE1FC,M544,2019-01-23,21/03/2021,NaN,False
4,1000034,23/01/2019,XE1FC,G570,2019-01-23,20/03/2021,NaN,False
...,...,...,...,...,...,...,...,...
133024,1027425,12/07/2012,J090.,E52XD,2012-12-07,21/03/2021,NaN,False
133025,1027425,12/07/2012,J090.,K938A,2012-12-07,09/01/2021,NaN,False
133026,1040468,NaN,NaN,NaN,NaT,30/01/2021,3.911491e+16,True
133027,1066390,NaN,NaN,NaN,NaT,29/01/2021,3.911491e+16,True


### add a column where it says if the event (event_dt_date_format) was before or after the vaccination (issue_date)

In [57]:
# convert issue date to date format
clinical_vac['issue_date_format'] = pd.to_datetime(clinical_vac['issue_date'])
clinical_vac
# renée: I guess it's safer, since the dtype is 'datetime64' in the new column and 'object' in the old one:
print(clinical_vac['issue_date']) 
print(clinical_vac['issue_date_format'])

0         23/02/2021
1         20/03/2021
2         20/03/2021
3         21/03/2021
4         20/03/2021
             ...    
133024    21/03/2021
133025    09/01/2021
133026    30/01/2021
133027    29/01/2021
133028    20/03/2021
Name: issue_date, Length: 133029, dtype: object
0        2021-02-23
1        2021-03-20
2        2021-03-20
3        2021-03-21
4        2021-03-20
            ...    
133024   2021-03-21
133025   2021-09-01
133026   2021-01-30
133027   2021-01-29
133028   2021-03-20
Name: issue_date_format, Length: 133029, dtype: datetime64[ns]


In [58]:
clinical_vac['event_dt_date_format'].iloc[0]>=clinical_vac['issue_date_format'].iloc[0]

False

In [59]:
before_after_vaccine = []
for i in range (len(clinical_vac)):
    if clinical_vac['event_dt_date_format'].iloc[i] >= clinical_vac['issue_date_format'].iloc[i]:
        # edit from monica: fill in the 'before/after' into the list, not overwrite the variable 
        before_after_vaccine.append('after')
    else:
        before_after_vaccine.append('before')
clinical_vac['before_after_vaccine'] = before_after_vaccine

In [60]:
clinical_vac

,eid,event_dt,code,icd10_code,event_dt_date_format,issue_date,dmd_code,vaccinated,issue_date_format,before_after_vaccine
0,1000034,01/01/1978,XE0e7,N419,1978-01-01,23/02/2021,NaN,False,2021-02-23,before
1,1000034,01/01/1978,XE0e7,N41,1978-01-01,20/03/2021,NaN,False,2021-03-20,before
2,1000034,23/01/2019,XE1FC,M543,2019-01-23,20/03/2021,NaN,False,2021-03-20,before
3,1000034,23/01/2019,XE1FC,M544,2019-01-23,21/03/2021,NaN,False,2021-03-21,before
4,1000034,23/01/2019,XE1FC,G570,2019-01-23,20/03/2021,NaN,False,2021-03-20,before
...,...,...,...,...,...,...,...,...,...,...
133024,1027425,12/07/2012,J090.,E52XD,2012-12-07,21/03/2021,NaN,False,2021-03-21,before
133025,1027425,12/07/2012,J090.,K938A,2012-12-07,09/01/2021,NaN,False,2021-09-01,before
133026,1040468,NaN,NaN,NaN,NaT,30/01/2021,3.911491e+16,True,2021-01-30,before
133027,1066390,NaN,NaN,NaN,NaT,29/01/2021,3.911491e+16,True,2021-01-29,before


In [61]:
# checking how many events before and after
clinical_vac['before_after_vaccine'].value_counts()

before    132440
after        589
Name: before_after_vaccine, dtype: int64

# Analysing clinical events

IC10 codes are structured. The following website was used for looking up the meanings of the categories (e.g. respiratory diseases):
https://www.icd10data.com/ICD10CM/Codes

In [62]:
ic10.head()

,Unnamed: 0,ICD10_CODE,ALT_CODE,USAGE,USAGE_UK,DESCRIPTION,MODIFIER_4,MODIFIER_5,QUALIFIERS,GENDER_MASK,MIN_AGE,MAX_AGE,TREE_DESCRIPTION
0,0,A00,A00,DEFAULT,3.0,Cholera,,,,,,,
1,1,A00.0,A000,DEFAULT,3.0,"Cholera due to Vibrio cholerae 01, biovar chol...",,,,,,,
2,2,A00.1,A001,DEFAULT,3.0,"Cholera due to Vibrio cholerae 01, biovar eltor",,,,,,,
3,3,A00.9,A009,DEFAULT,3.0,"Cholera, unspecified",,,,,,,
4,4,A01,A01,DEFAULT,3.0,Typhoid and paratyphoid fevers,,,,,,,


the GP_ic10 has the read code which is the CTV3. 
(However, when things get nasty, there are multiple ICD10 codes per CTV3 codes and viceversa… there is a mapping status which tells you whether the code is primary, duplicated, to revision, etc)

In [63]:
# WILL PROBABLY NOT USE THIS

# just to see if corresponding CTV3 to IC10 works

# broadcasting for vaccinated comorbidities(CTV3) in ICD10
mask_vac_ic10 = GPic10['read_code'].isin(clinical_ic10[mask_vac_clin]['code']) 
GPic10[mask_vac_ic10]

# mask_vac: people that got any vaccine
# search for patients that got the vaccine in GPic10
# and look what clinical events they had via the read_code (e.g. XE01qn)


,Unnamed: 0,read_code,icd10_code,mapping_status,refine_flag,add_code_flag,element_num,block_num
561,561,1712.,R05X,G,C,C,0.0,0.0
566,566,1719.,R05X,G,C,C,0.0,0.0
596,596,182..,R074,D,C,C,0.0,0.0
597,597,182..,R071,A,C,C,0.0,0.0
598,598,182..,R072,A,C,C,0.0,0.0
...,...,...,...,...,...,...,...,...
107037,107037,Xa7Ni,L089,G,C,C,0.0,0.0
111860,111860,XaFr9,R030,G,C,C,0.0,0.0
112328,112328,XaIfN,L82X,G,C,C,0.0,0.0
114868,114868,Xabdb,R53X,D,C,C,0.0,0.0


if time: analyse specific diseases such as myocarditis etc.

In [64]:
clinical_ic10

,eid,event_dt,code,icd10_code,event_dt_date_format
12,1000034,01/01/1978,XE0e7,N419,1978-01-01
13,1000034,01/01/1978,XE0e7,N41,1978-01-01
14,1013867,10/01/2003,XE0e7,N419,2003-10-01
15,1013867,10/01/2003,XE0e7,N41,2003-10-01
16,1000034,23/01/2019,XE1FC,M543,2019-01-23
...,...,...,...,...,...
188460,1033266,11/02/2016,XaQv0,R452,2016-11-02
188461,1033266,25/02/2015,X75Z0,R568,2015-02-25
188488,1033266,23/06/2015,XaDbE,R568,2015-06-23
188489,1033328,11/12/2018,X2060,I839,2018-11-12


In [65]:
ic10.head(3)

,Unnamed: 0,ICD10_CODE,ALT_CODE,USAGE,USAGE_UK,DESCRIPTION,MODIFIER_4,MODIFIER_5,QUALIFIERS,GENDER_MASK,MIN_AGE,MAX_AGE,TREE_DESCRIPTION
0,0,A00,A00,DEFAULT,3.0,Cholera,,,,,,,
1,1,A00.0,A000,DEFAULT,3.0,"Cholera due to Vibrio cholerae 01, biovar chol...",,,,,,,
2,2,A00.1,A001,DEFAULT,3.0,"Cholera due to Vibrio cholerae 01, biovar eltor",,,,,,,


In [66]:
# mask for getting codes related to myocarditis
mask_myocarditis = ic10['DESCRIPTION'].str.contains('myocarditis', case = False)
ic10[mask_myocarditis]

,Unnamed: 0,ICD10_CODE,ALT_CODE,USAGE,USAGE_UK,DESCRIPTION,MODIFIER_4,MODIFIER_5,QUALIFIERS,GENDER_MASK,MIN_AGE,MAX_AGE,TREE_DESCRIPTION
3700,3700,I01.2,I012,DEFAULT,3.0,Acute rheumatic myocarditis,,,,,,,
3732,3732,I09.0,I090,DEFAULT,3.0,Rheumatic myocarditis,,,,,,,
3861,3861,I40,I40,DEFAULT,3.0,Acute myocarditis,,,,,,,
3862,3862,I40.0,I400,DEFAULT,3.0,Infective myocarditis,,,,,,,
3863,3863,I40.1,I401,DEFAULT,3.0,Isolated myocarditis,,,,,,,
3864,3864,I40.8,I408,DEFAULT,3.0,Other acute myocarditis,,,,,,,
3865,3865,I40.9,I409,DEFAULT,3.0,"Acute myocarditis, unspecified",,,,,,,
3866,3866,I41,I41,ASTERIS,2.0,Myocarditis in diseases classified elsewhere,,,,,,,
3867,3867,I41.0,I410,ASTERIS,2.0,Myocarditis in bacterial diseases classified e...,,,,,,,
3868,3868,I41.1,I411,ASTERIS,2.0,Myocarditis in viral diseases classified elsew...,,,,,,,


In [67]:
# getting data with myocarditis

# the alternative code 'ALT_CODE' used here, as this was used in the lookup table GPic10
myocarditis = ic10[mask_myocarditis]['ALT_CODE']

clinical_ic10[clinical_ic10['icd10_code'].isin(myocarditis)]
# nobody had myocarditis in this restricted data

,eid,event_dt,code,icd10_code,event_dt_date_format


In [68]:
## ishaemic heart disease
mask_ihd = ic10['DESCRIPTION'].str.contains('ischaemic heart disease', case = False)
ischaemicHD = ic10[mask_ihd]['ALT_CODE']
clinical_ic10[clinical_ic10['icd10_code'].isin(ischaemicHD)]

,eid,event_dt,code,icd10_code,event_dt_date_format
56090,1000118,07/02/2014,XaINF,I249,2014-07-02
56091,1000118,15/12/2013,XaINF,I249,2013-12-15
56092,1000118,14/10/2014,XaINF,I249,2014-10-14
56093,1000912,31/10/2008,XaINF,I249,2008-10-31
56094,1011364,05/03/2015,XaINF,I249,2015-05-03
...,...,...,...,...,...
180660,1013003,25/11/2013,G3y..,I258,2013-11-25
182750,1016004,02/09/2010,XaG1Q,I259,2010-02-09
182751,1016004,02/09/2010,XaG1Q,I25,2010-02-09
184395,1019090,18/01/1994,G20..,I24,1994-01-18


several comorbidities

In [69]:
# getting people with specific diseases

# new column for filling with selected disease
clinical_ic10['selected disease'] = np.nan 

# list of specific diseases, to be decided on
diseases_selec = ['ischaemic heart disease', 'cardiomyopathy', 'heart failure',\
                  'myocarditis'] 

for disease in diseases_selec:
    
    # broadcasting for selected disease
    mask = ic10['DESCRIPTION'].str.contains(disease, case = False) 
    # broadcasting for selected disease
    sub_disease = clinical_ic10[clinical_ic10['icd10_code'].isin(ic10[mask]['ALT_CODE'])]
    
    for i in sub_disease.index:
        # fill dataset with selected disease label
        clinical_ic10.loc[i,'selected disease'] = disease 

C:\Users\moniz\AppData\Local\Temp/ipykernel_7584/506691620.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clinical_ic10['selected disease'] = np.nan
C:\Users\moniz\AppData\Roaming\Python\Python38\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [70]:
# counting how many people in the dataset have a specific disease
clinical_ic10['selected disease'].value_counts()

ischaemic heart disease    680
heart failure              188
cardiomyopathy              57
Name: selected disease, dtype: int64

# getting people w. different categories of underlying diseases

# Question 1: What is the effect on adverse events?
## 1. How common are "adverse events"? (i.e. how many percent of vaccinated people have registered medical events after the vaccine) compare vaccinated & nonvaccinated
## 2. What about people with medical history?
## 3. Are they more common in people with (specific) underlying medical conditions? i.e.: 
* plot percentage of AEs with & without underlying conditions
* plot percentages for specific underlying conditions (not considering if several conditions at once, plot for each category then), for vaccinated and non-vaccinated people
* (optional: just for vaccinated people: plot what types of AEs (in IC10 categories) are common.

In [71]:
clinical_vac

,eid,event_dt,code,icd10_code,event_dt_date_format,issue_date,dmd_code,vaccinated,issue_date_format,before_after_vaccine
0,1000034,01/01/1978,XE0e7,N419,1978-01-01,23/02/2021,NaN,False,2021-02-23,before
1,1000034,01/01/1978,XE0e7,N41,1978-01-01,20/03/2021,NaN,False,2021-03-20,before
2,1000034,23/01/2019,XE1FC,M543,2019-01-23,20/03/2021,NaN,False,2021-03-20,before
3,1000034,23/01/2019,XE1FC,M544,2019-01-23,21/03/2021,NaN,False,2021-03-21,before
4,1000034,23/01/2019,XE1FC,G570,2019-01-23,20/03/2021,NaN,False,2021-03-20,before
...,...,...,...,...,...,...,...,...,...,...
133024,1027425,12/07/2012,J090.,E52XD,2012-12-07,21/03/2021,NaN,False,2021-03-21,before
133025,1027425,12/07/2012,J090.,K938A,2012-12-07,09/01/2021,NaN,False,2021-09-01,before
133026,1040468,NaN,NaN,NaN,NaT,30/01/2021,3.911491e+16,True,2021-01-30,before
133027,1066390,NaN,NaN,NaN,NaT,29/01/2021,3.911491e+16,True,2021-01-29,before


Get number of adverse events (defined medical events w. ICD10 code after the (artificial) vaccination date)

In [95]:
# reminder: leave R as adverse event & Q as underlying disease.

# subset of clinical dataset for adverse events (=after vaccine)
df_AEs = clinical_vac[clinical_vac['before_after_vaccine']== 'after']

# remove Q (congenital diseases), which are not considered as adverse events
df_AEs = df_AEs[~df_AEs['icd10_code'].str.startswith('Q')]

# number of AEs in this dataset
print('total number of AEs:', len(df_AEs))

total number of AEs: 582


In [96]:
# number of AEs in vaccinated people
AE_vac_all = len(df_AEs[df_AEs['vaccinated']==True])
print('total number of AEs in vaccinated people:', AE_vac_all)

total number of AEs in vaccinated people: 0


In [97]:
# number of AEs in non-vaccinated people
AE_nvac_all = len(df_AEs[df_AEs['vaccinated']==False])
print('total number of AEs in non-vaccinated people:', AE_nvac_all)

total number of AEs in non-vaccinated people: 582


Get number of people available in the dataset(s).

# Unused code so far

In [73]:
# code layout for getting the different categories

# subsetting into categories using IC10
# e.g. I00-I99 Diseases of the circulatory system, more details available too, e.g. Ischemic heart diseases
GPic10.dropna(inplace=True) # drop NaN to enable making mask

m_circ = GPic10['icd10_code'].str.startswith('I')
GPic10[m_circ]['read_code']

302       14A3.
303       14A4.
600       182..
602       1822.
610       1828.
          ...  
115198    Xafis
115199    Xafis
115200    Xafis
115204    Xafyq
115207    Xafyq
Name: read_code, Length: 3771, dtype: object

In [74]:
# getting corresponding CTV3 codes
m_circ_CTV3 = code['coding'].isin(GPic10[m_circ]['read_code'])
code[m_circ_CTV3]

,coding,meaning
34065,14A3.,H/O: myocardial infarct at less than 60
34066,14A4.,H/O: myocardial infarct at greater than 60
34881,182..,Chest pain
34883,1822.,Central chest pain
34889,1828.,Atypical chest pain
...,...,...
180522,XM1Qq,Thrombophlebitis
180523,XM1Qr,Superficial vein thrombosis
180524,XM1Qs,Perineal varices
180526,XM1Qu,Claudication


In [75]:
 # getting subjects with this condition
m_circ_sub = clinical['code'].isin(code[m_circ_CTV3]['coding']) # another mask

# correspond to vaccinated or not

In [76]:
# no one with any vaccine had circulatory issue before vaccine on this limited dataset
scripts[mask_vac]['eid'].isin(clinical[m_circ_sub]['eid']).sum()

2